In [ ]:
DATASET_PATH = "/ssd/datasets/argoverse/argoverse-forecasting-dataset/"
TRAIN = DATASET_PATH + "train/data"
VAL = DATASET_PATH + "val/data"
TEST = DATASET_PATH + "test_obs/data/"

In [ ]:
import os
from torch.utils.data import Dataset
from pathlib import Path
import pandas as pd
import numpy as np

class TrajectoryDataset(Dataset):
    
    def __init__(self, root_dir, mode):
        self.root_dir = Path(root_dir)
        self.mode = mode
        self.sequences = [(self.root_dir / x).absolute() for x in os.listdir(self.root_dir)]
        self.obs_len = 20
        
    def __len__(self):
        return len(self.sequences)
    
    def __getitem__(self, idx):
        sequence = pd.read_csv(self.sequences[idx])
        agent_x = sequence[sequence["OBJECT_TYPE"] == "AGENT"]["X"]
        agent_y = sequence[sequence["OBJECT_TYPE"] == "AGENT"]["Y"]
        agent_traj = np.column_stack((agent_x, agent_y))
        return {"input": agent_traj[:self.obs_len], "target": agent_traj[self.obs_len:]}

In [ ]:
train_dataset = TrajectoryDataset(TRAIN, "train")

In [ ]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=6)

In [ ]:
for i, batch in enumerate(train_dataloader):
    if i > 10:
        break
    print(i, batch["input"].size(), batch["target"].size())
    print(batch["input"])

In [ ]:
for i in range(3):
    sample = train_dataset[i]
    observed = sample["input"]
    prediction = sample["output"]